In [70]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt


from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass
from alpaca.data.requests import StockBarsRequest
from alpaca.data.requests import StockQuotesRequest
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.timeframe import TimeFrame

In [124]:
#Generate the trading signal 
def signal_volume(stock_data, symbol, date, V50, i, request):
    day = date.date()
    start_time = pd.Timestamp('10:00:00').to_pydatetime().time()
    start_datetime = datetime.combine(day, start_time)
    end_time = pd.Timestamp('17:00:00').to_pydatetime().time()
    end_datetime = datetime.combine(day, end_time)

    
    timeframe = TimeFrame.Hour  

    # Define the request parameters
    request_params = StockBarsRequest(
        symbol_or_symbols=[symbol],  
        timeframe=timeframe,  
        start=start_datetime 
    )
    hourly_data = data_client.get_stock_bars(request_params).df

    #get the stock data for the day
    hourly_data.reset_index(inplace=True)

    volume_percentage = 0.6

    for time in hourly_data['timestamp']:
        curr_volume = hourly_data.loc[hourly_data["timestamp"] == time, "volume"].values

        if(volume_percentage * curr_volume >= V50):
            stock_data.loc[i, 'signal'] = request
            return
        volume_percentage += 0.2





#Identify if there is a moving average crossover between the two 
def signal_crossover(stock_data, symbol):
    """    
    Buy signal: Closing price crosses above the 200-day moving average. [1]
    Sell signal: Closing price crosses below the 200-day moving average. [-1]
    Keep signal: None of the above conditions apply, no action is performed with the stock [0]
    """
    # Generate crossover signal
    print(symbol)
    for i in range(1, len(stock_data)):
        if pd.isna(stock_data.loc[i, '50DMA']) or pd.isna(stock_data.loc[i, '200DMA']):
            continue
        if stock_data.loc[i, '50DMA'] > stock_data.loc[i, '200DMA'] and stock_data.loc[i - 1, '50DMA'] <= stock_data.loc[i - 1, '200DMA']:
            print(symbol)
            print(stock_data)
            print(f"buy: 50D: {stock_data.loc[i, '50DMA']}  200D: {stock_data.loc[i, '200DMA']}")
            signal_volume(stock_data, symbol, stock_data.loc[i, 'timestamp'], stock_data.loc[i, "50DVMA"], i, 1)
            # stock_data.loc[i, 'signal'] = 1  # Buy signal
        elif stock_data.loc[i, '50DMA'] < stock_data.loc[i, '200DMA'] and stock_data.loc[i - 1, '50DMA'] >= stock_data.loc[i - 1, '200DMA']:
            signal_volume(stock_data, symbol, stock_data.loc[i, 'timestamp'], stock_data.loc[i, "50DVMA"], i, -1)
            # stock_data.loc[i, 'signal'] = -1  # Sell signal



# Generate trading signals based on the 200-day moving average
def signal_generator(stock_data, symbol):
    """
    Generates trading signals based on the 200-day moving average and volume.
    Buy signal: Closing price crosses above the 200-day moving average. [1]
    Sell signal: Closing price crosses below the 200-day moving average. [-1]
    Keep signal: None of the above conditions apply, no action is performed with the stock [0]
    """
    stock_data['200DMA'] = stock_data['close'].rolling(window=200).mean()
    stock_data['50DMA'] = stock_data['close'].rolling(window=50).mean()
    stock_data['50DVMA'] = stock_data['volume'].rolling(window=50).mean()

    stock_data['signal'] = 0
    signal_crossover(stock_data, symbol)

    return stock_data

def trading_job(symbol, bars, portfolio):
    """
    Executes trades based on signals generated from the 200-day moving average strategy.
    """
    # Request historical data for the given symbol

    # Process and generate signals
    df = pd.DataFrame(bars.loc[symbol])  # Extract data for the specific symbol
    df.reset_index(inplace=True)
    df = signal_generator(df, symbol)
    # Execute trades based on the signal
    for i in range(len(df)):
        if df['signal'].iloc[i] == 1:
            print(f"Buy signal generated for {symbol} on {df['timestamp'][i]} at price {df['close'][i]}")
        elif df['signal'].iloc[i] == -1 and symbol in portfolio:
            print(f"Sell signal generated for {symbol} on {df['timestamp']} at price {df['close'][i]}")

# Run the trading job for specific symbols
#snp500 = pd.read_csv("constituents.csv")
#tickers = snp500["Symbol"].to_list()
if False:
    tickers = ["AMCR", "AMT", "AWK", "ADI", "BWA", "FTV", "ODFL"]
    request_params = StockBarsRequest(
        symbol_or_symbols=tickers,
        timeframe=TimeFrame.Day,
        start=datetime.now(tz=None) - timedelta(days=300),
    )
    bars = data_client.get_stock_bars(request_params).df

portfolio = []
for symbol in tickers:
    trading_job(symbol, bars, portfolio)

AMCR
AMT
AWK
ADI
BWA
FTV
FTV
                    timestamp   open    high     low  close     volume  \
0   2024-03-12 04:00:00+00:00  85.00  86.395  84.470  86.20  1588118.0   
1   2024-03-13 04:00:00+00:00  86.23  86.750  85.100  85.46  1371372.0   
2   2024-03-14 04:00:00+00:00  86.16  86.490  85.020  85.78  1461972.0   
3   2024-03-15 04:00:00+00:00  84.80  85.730  84.182  85.35  2159040.0   
4   2024-03-18 04:00:00+00:00  85.76  85.990  85.170  85.22  1104375.0   
..                        ...    ...     ...     ...    ...        ...   
201 2024-12-27 05:00:00+00:00  75.50  76.130  75.195  75.65  2350118.0   
202 2024-12-30 05:00:00+00:00  74.85  75.240  74.050  75.04  1703122.0   
203 2024-12-31 05:00:00+00:00  75.32  75.850  74.820  75.00  1227969.0   
204 2025-01-02 05:00:00+00:00  75.46  75.690  74.280  74.41  1870347.0   
205 2025-01-03 05:00:00+00:00  74.47  75.990  74.345  75.90  1726001.0   

     trade_count       vwap    200DMA    50DMA      50DVMA  signal  
0        2283

In [162]:
def get_stock_data(stock_symbols):
    # Get today's date and calculate the date one year ago
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)
    
    stock_data = {}
    
    for symbol in stock_symbols:
        # Fetch data from Yahoo Finance
        stock_df = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
        
        if not stock_df.empty:
            stock_data[symbol] = stock_df
        else:
            print(f"No data found for {symbol}")
    
    return stock_data

# List of stock symbols
stocks = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]  # Example list of stocks

# Get stock data for the last year
data = get_stock_data(stocks)

# Example: Access data for a specific stock
for symbol, df in data.items():
    print(f"\nData for {symbol}:")
    print(df.head())  # Print the first few rows of the DataFrame

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Data for AAPL:

Data for MSFT:

Data for GOOGL:

Data for AMZN:

Data for TSLA:
